# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 8:30AM,243311,10,0085921540,ISDIN Corporation,Released
1,Aug 3 2022 8:30AM,243311,10,0085921557,ISDIN Corporation,Released
2,Aug 3 2022 8:30AM,243311,10,0085921554,ISDIN Corporation,Released
3,Aug 3 2022 8:30AM,243311,10,0085921605,ISDIN Corporation,Released
4,Aug 3 2022 8:30AM,243311,10,0085921604,ISDIN Corporation,Released
5,Aug 3 2022 8:30AM,243311,10,0085921606,ISDIN Corporation,Released
6,Aug 3 2022 8:30AM,243311,10,0085921611,ISDIN Corporation,Released
7,Aug 3 2022 8:30AM,243311,10,0085921569,ISDIN Corporation,Released
8,Aug 3 2022 8:30AM,243311,10,0085921597,ISDIN Corporation,Released
9,Aug 2 2022 3:19PM,243309,10,8352817,Yusen – 3D Matrix,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,243303,Released,1
36,243304,Released,2
37,243308,Released,1
38,243309,Released,1
39,243311,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243303,NaN,NaN,1.0
243304,NaN,NaN,2.0
243308,NaN,NaN,1.0
243309,NaN,NaN,1.0
243311,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243174,14.0,21.0,2.0
243210,5.0,6.0,2.0
243217,0.0,0.0,1.0
243218,0.0,1.0,0.0
243241,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243174,14,21,2
243210,5,6,2
243217,0,0,1
243218,0,1,0
243241,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243174,14,21,2
1,243210,5,6,2
2,243217,0,0,1
3,243218,0,1,0
4,243241,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243174,14,21,2
1,243210,5,6,2
2,243217,,,1
3,243218,,1,
4,243241,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation
9,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix
10,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix
11,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix
12,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC"
14,Aug 2 2022 3:04PM,243303,10,"Snap Medical Industries, LLC"
15,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC"
16,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions
17,Aug 2 2022 2:56PM,243300,16,American International Chemical
18,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,,,9
1,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,,,1
2,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix,,,1
3,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix,,,1
4,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC",,,2
5,Aug 2 2022 3:04PM,243303,10,"Snap Medical Industries, LLC",,,1
6,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",,,1
7,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,,,1
8,Aug 2 2022 2:56PM,243300,16,American International Chemical,,,1
9,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,
1,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,1,,
2,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix,1,,
3,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix,1,,
4,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC",2,,
5,Aug 2 2022 3:04PM,243303,10,"Snap Medical Industries, LLC",1,,
6,Aug 2 2022 3:00PM,243302,10,"Snap Medical Industries, LLC",1,,
7,Aug 2 2022 2:58PM,243301,16,Sartorius Bioprocess Solutions,1,,
8,Aug 2 2022 2:56PM,243300,16,American International Chemical,1,,
9,Aug 2 2022 2:38PM,243298,10,ISDIN Corporation,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,
1,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,1,,
2,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix,1,,
3,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix,1,,
4,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,NaN,NaN
1,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,0.0,0.0
1,Aug 2 2022 3:19PM,243309,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Aug 2 2022 3:16PM,243308,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Aug 2 2022 3:11PM,243270,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Aug 2 2022 3:09PM,243304,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5595597,115.0,26.0,6.0
15,243174,2.0,21.0,14.0
16,486601,2.0,0.0,0.0
19,243241,0.0,1.0,0.0
20,972948,4.0,28.0,27.0
21,486477,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5595597,115.0,26.0,6.0
1,15,243174,2.0,21.0,14.0
2,16,486601,2.0,0.0,0.0
3,19,243241,0.0,1.0,0.0
4,20,972948,4.0,28.0,27.0
5,21,486477,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,115.0,26.0,6.0
1,15,2.0,21.0,14.0
2,16,2.0,0.0,0.0
3,19,0.0,1.0,0.0
4,20,4.0,28.0,27.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,115.0
1,15,Released,2.0
2,16,Released,2.0
3,19,Released,0.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,6.0,14.0,0.0,0.0,27.0,0.0
Executing,26.0,21.0,0.0,1.0,28.0,2.0
Released,115.0,2.0,2.0,0.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,6.0,14.0,0.0,0.0,27.0,0.0
1,Executing,26.0,21.0,0.0,1.0,28.0,2.0
2,Released,115.0,2.0,2.0,0.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,6.0,14.0,0.0,0.0,27.0,0.0
1,Executing,26.0,21.0,0.0,1.0,28.0,2.0
2,Released,115.0,2.0,2.0,0.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()